In [1]:
import cv2

def crop_and_save_image(image_path, new_width, new_height, output_path):
    # Charger l'image
    image = cv2.imread(image_path)

    # Obtenir les dimensions actuelles de l'image
    height, width, _ = image.shape

    # Calculer le ratio de redimensionnement
    target_ratio = new_width / new_height
    current_ratio = width / height

    if current_ratio < target_ratio:
        # Recadrer l'image en retirant une partie de l'image en haut et en bas
        target_height = int(width / target_ratio)
        crop_start = int((height - target_height) / 2)
        crop_end = int(crop_start + target_height)
        cropped_image = image[crop_start:crop_end, :]
    else:
        # Recadrer l'image en retirant une partie de l'image à gauche et à droite
        target_width = int(height * target_ratio)
        crop_start = int((width - target_width) / 2)
        crop_end = int(crop_start + target_width)
        cropped_image = image[:, crop_start:crop_end]

    # Redimensionner l'image aux nouvelles dimensions
    resized_image = cv2.resize(cropped_image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Sauvegarder l'image redimensionnée à l'emplacement spécifié
    cv2.imwrite(output_path, resized_image)



In [9]:
import cv2
import numpy as np

def remove_background(image_path, output_path):
    # Charger l'image
    image = cv2.imread(image_path)

    # Obtenir les dimensions de l'image
    height, width, _ = image.shape

    # Créer un masque avec une zone blanche à gauche et à droite de la diagonale
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, np.array([[(0, 0), (width, 0), (width, height), (0, height)]]), 255)
    cv2.fillPoly(mask, np.array([[(0, height), (width, 0), (width, height)]]), 0)

    # Appliquer le masque sur l'image
    result = cv2.bitwise_and(image, image, mask=mask)

    # Sauvegarder l'image résultante à l'emplacement spécifié
    cv2.imwrite(output_path, result)

# Exemple d'utilisation
image_path = 'data/resized_images/cropped_Billy_Butcher.jpg'
output_path = 'data/resized_images/cropped_Billy_Butcher2.jpg'

remove_background(image_path, output_path)

In [21]:
import cv2
import numpy as np

def remove_background(image_path, output_path, diagonal_coords):
    # Charger l'image
    image = cv2.imread(image_path)

    # Créer un masque d'image avec des pixels opaques (canal alpha à 255)
    mask = np.ones_like(image[:, :, 0], dtype=np.uint8) * 255

    # Tracer la diagonale et marquer les pixels correspondants dans le masque comme transparents
    cv2.drawContours(mask, [np.array(diagonal_coords)], -1, 0, thickness=cv2.FILLED)

    # Séparer les canaux B, G, R de l'image d'origine
    b, g, r = cv2.split(image)

    # Convertir les canaux en format RGBA en ajoutant un canal alpha
    bgra = cv2.merge((b, g, r, mask))

    # Enregistrer l'image résultante au format PNG avec transparence
    cv2.imwrite(output_path, bgra)

In [122]:
# Exemple d'utilisation
image_path = 'data/resized_images/cropped_Billy_Butcher.jpg'
output_path = 'data/resized_images/sliced_Billy_Butcher.png'

# Coordonnées des pixels pour tracer la diagonale (exemple)
height, width, _ = cv2.imread(image_path).shape
diagonal_coords = [[width, 0], [width-250, height], [width, height]]

remove_background(image_path, output_path, diagonal_coords)

In [123]:
# Exemple d'utilisation
image_path = 'data/resized_images/cropped_Gustavo_Fring.jpg'
output_path = 'data/resized_images/sliced_Gustavo_Fring.png'

# Coordonnées des pixels pour tracer la diagonale (exemple)
height, width, _ = cv2.imread(image_path).shape
diagonal_coords = [[0, 0], [250, 0], [0, height]]

remove_background(image_path, output_path, diagonal_coords)

In [3]:
from PIL import Image

def combiner_images(image1_path, image2_path, model_path, output_path, superposition):
    # Charger les images
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    model = Image.open(model_path)

    # Récupérer les dimensions des images
    largeur_image1, hauteur_image1 = image1.size
    largeur_image2, hauteur_image2 = image2.size

    # Déterminer les dimensions du fond blanc
    hauteur_fond = max(hauteur_image1, hauteur_image2)
    largeur_fond = largeur_image1 + largeur_image2 - superposition

    # Créer une nouvelle image avec le fond blanc
    fond = Image.new('RGBA', (largeur_fond, hauteur_fond), (255, 255, 255, 255))

    # Placer la première image à gauche
    fond.paste(image1, (0, 0), mask=image1)

    # Placer la deuxième image à droite
    fond.paste(image2, (largeur_image1-superposition, 0), mask=image2)

    # Placement du modèle
    fond.paste(model, (largeur_image1-superposition+2, 0), mask=model)

    # Sauvegarder l'image combinée
    fond.save(output_path)



In [4]:
# Exemple d'utilisation
image1_path = 'data/resized_images/sliced_Billy_Butcher.png'
image2_path = 'data/resized_images/sliced_Gustavo_Fring.png'
model_path = 'data/models/versus_model_1.png'
output_path = 'data/resized_images/result.png'

combiner_images(image1_path, image2_path, model_path, output_path, 254)